## Setup

In [68]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [69]:
import pandas as pd
from pydantic import BaseModel
import sys
import os
from lightning.pytorch.loggers import MLFlowLogger
from loguru import logger
from load_dotenv import load_dotenv
import torch
from torch.utils.data import DataLoader
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
import mlflow

sys.path.insert(0, "..")

from src.utils.embedding_id_mapper import IDMapper
from src.algo.sequence.model import SequenceRatingPrediction
from src.algo.sequence.dataset import UserItemBinaryRatingDFDataset
from src.algo.sequence.trainer import SeqModellingLitModule
from src.eval.utils import create_rec_df, create_label_df, merge_recs_with_target
from src.eval.log_metrics import log_ranking_metrics, log_classification_metrics

In [70]:
load_dotenv(override = True)

True

In [71]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "first-attempt"
    notebook_persit_dp: str = None
    
    run_name: str = None

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"
    group_name: str = "seq-modelling"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128
    learning_rate: float = 0.001
    l2_reg: float = 1e-6
    early_stopping_patience: int = 10
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    max_epochs: int = 100

    # TwoTower specific
    dropout: float = 0.3
    embedding_dim: int = 256


    train_data_fp: str = os.path.abspath("../data_for_ai/interim/train_sample_interactions_16407u_neg_seq_2.parquet")
    val_data_fp: str = os.path.abspath("../data_for_ai/interim/val_sample_interactions_16407u_neg_seq_2.parquet")

    best_checkpoint_path: str = None
    def init(self):
        self.run_name: str = f"006-sequence-modelling-attn-{self.embedding_dim}-dim-bce-prelu"
        self.notebook_persit_dp = os.path.abspath(f"data/{self.experiment_name}/{self.run_name}")

        if not (mlflow_uri := os.environ.get("MLFLOW_TRACKING_URI")):
            self.log_to_mlflow = False
            logger.warning("MLFlow is not enabled. Turn off tracking to Mlflow.")

        if self.log_to_mlflow:
            logger.info(
                f"Setting up Mlflow experiment: {self.experiment_name}, run_name: {self.run_name}"
            )

            self._mlf_logger = MLFlowLogger(
                experiment_name=self.experiment_name,
                run_name=self.run_name,
                tracking_uri=mlflow_uri,
                log_model=True,
            )

        if not self.testing:
            os.makedirs(self.notebook_persit_dp, exist_ok=True)
        return self
    
args = Args().init()
print(args.model_dump_json(indent=2))

2025-06-21 03:02:33.141 | INFO     | __main__:init:43 - Setting up Mlflow experiment: first-attempt, run_name: 006-sequence-modelling-attn-256-dim-bce-prelu


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "first-attempt",
  "notebook_persit_dp": "/home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/006-sequence-modelling-attn-256-dim-bce-prelu",
  "run_name": "006-sequence-modelling-attn-256-dim-bce-prelu",
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "group_name": "seq-modelling",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128,
  "learning_rate": 0.001,
  "l2_reg": 1e-6,
  "early_stopping_patience": 10,
  "device": "cuda",
  "max_epochs": 100,
  "dropout": 0.3,
  "embedding_dim": 256,
  "train_data_fp": "/home/dinhln/Desktop/real_time_recsys/data_for_ai/interim/train_sample_interactions_16407u_neg_seq_2.parquet",
  "val_data_fp": "/home/dinhln/Desktop/real_time_recsys/data_for_ai/interim/val_sample_interactions_16407u_neg_seq_2.parquet",
  "best_checkpoint_path": null
}


## Init model

In [72]:
def init_model(n_users, n_items, embedding_dim, dropout, item_embedding=None):
    return SequenceRatingPrediction(
        item_embedding=item_embedding,
        num_users=n_users,
        num_items=n_items,
        embedding_dim=embedding_dim,
        dropout=dropout,
    )

## Test implementation

In [73]:
embedding_dim = 32
batch_size = 2

# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
timestamps = [0, 1, 2, 3, 4]
ratings = [0, 3, 1, 3, 0]
# item_sequences = [
#     [2, 3, -1, -1],
#     [2, 4, -1, -1],
#     [1, 3, -1, -1],
#     [2, 1, -1, -1],
#     [4, 1, -1, -1],
# ]

item_sequences = [
    [-1, -1, 2, 3],
    [-1, -1, 2, 4],
    [-1, -1, 1, 3],
    [-1, -1, 2, 1],
    [-1, -1, 4, 1],
]


n_users = len(set(user_indices))

n_items = len(set(item_indices))

train_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.timestamp_col: timestamps,
        args.rating_col: ratings,
        "item_sequence": item_sequences,
    }
)

model = init_model(n_users, n_items, embedding_dim, args.dropout)

# Example forward pass
model.eval()
user = torch.tensor([0])
item_sequence = torch.tensor([[-0, 1, -1, -1]])
target_item = torch.tensor([2])
predictions = model(user, item_sequence, target_item)
print(predictions)
model.train()

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:385: UserWarning:

enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.activation_relu_or_gelu was not True

2025-06-21 03:02:33.411 | INFO     | src.algo.sequence.model:__init__:118 - Start token used: 4, Padding token used: 5


tensor([[0.0779]], grad_fn=<MaskedFillBackward0>)


SequenceRatingPrediction(
  (item_embedding): Embedding(6, 32, padding_idx=5)
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
    )
    (linear1): Linear(in_features=32, out_features=32, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (linear2): Linear(in_features=32, out_features=32, bias=True)
    (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.3, inplace=False)
    (dropout2): Dropout(p=0.3, inplace=False)
    (activation): PReLU(num_parameters=1)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_feat

In [74]:
rating_dataset = UserItemBinaryRatingDFDataset(
    train_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col,"item_sequence"
)

train_loader = DataLoader(
    rating_dataset, batch_size=batch_size, shuffle=False, drop_last=True
)

In [75]:
for batch_input in train_loader:
    print(batch_input)

{'user': tensor([0, 0]), 'item': tensor([0, 1]), 'rating': tensor([0., 1.]), 'item_sequence': tensor([[-1, -1,  2,  3],
        [-1, -1,  2,  4]], dtype=torch.int32)}
{'user': tensor([1, 2]), 'item': tensor([2, 3]), 'rating': tensor([1., 1.]), 'item_sequence': tensor([[-1, -1,  1,  3],
        [-1, -1,  2,  1]], dtype=torch.int32)}


In [76]:
# model
lit_model = SeqModellingLitModule(model, log_dir=args.notebook_persit_dp)

# train model
trainer = L.Trainer(
    default_root_dir=f"{args.notebook_persit_dp}/test",
    max_epochs=100,
    accelerator=args.device if args.device else "auto",
)
trainer.fit(
    model=lit_model, train_dataloaders=train_loader, val_dataloaders=train_loader
)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type                     | Params | Mode 
------------------------------------------------------------------------
0 | model              | SequenceRatingPrediction | 16.4 K | train
1 | val_roc_auc_metric | BinaryAUROC              | 0      | train
2 | val_pr_auc_metric  | BinaryAveragePrecision   | 0      | train
------------------------------------------------------------------------
16.4 K    Trainable params
0         Non-trainable params
16.4 K    Total params
0.065     Total estimated model params size (MB)
33        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: PossibleUserWarning:

The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you 

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [77]:
model.eval()
user = torch.tensor([0])
item_sequence = torch.tensor([[-1, -1, -1, 0, 1]])
target_item = torch.tensor([2])
predictions = model.predict(user, item_sequence, target_item)
print(predictions)

tensor([[0.9969]], grad_fn=<SigmoidBackward0>)


## Training loop

In [78]:
train_df = pd.read_parquet(args.train_data_fp)
val_df = pd.read_parquet(args.val_data_fp)

assert set(val_df[args.user_col].unique()).issubset(set(train_df[args.user_col].unique())), "Validation users must be present in training users."

assert set(val_df[args.item_col].unique()).issubset(set(train_df[args.item_col].unique())), "Validation items must be present in training items."
assert train_df[args.timestamp_col].max() < val_df[args.timestamp_col].min(), "Validation data must be after training data. Otherwise, its a data contamination problem."

In [79]:
train_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
266890,AETCSNOCHP5YL3C67YWXHB54RCNA,B09BW3XJQV,0.0,2016-11-18 05:32:16.000,3257,1248,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 142..."
183685,AHUDTHSVE5XAFVPOHHBSPL7T7TNA,B00B3OA7R8,0.0,2014-11-23 14:06:52.000,15687,2394,"[-1.0, -1.0, -1.0, -1.0, 3375.0, 3642.0, 954.0..."
308523,AGYLXGPK4SKKQZ2OV3LXYGHFYM2A,B0895KGSY1,0.0,2017-12-31 06:21:06.077,12067,4591,"[-1.0, -1.0, -1.0, -1.0, -1.0, 4458.0, 1570.0,..."
99169,AENO3VFOX37KYSKS6FQOXWSHR74Q,B078CQSHDV,5.0,2018-08-06 20:46:04.598,2540,3168,"[-1.0, -1.0, -1.0, 650.0, 2326.0, 3089.0, 2787..."
64744,AHGMLXGDYDPBY3MIJZPHUAOVZFVQ,B08NW1RXPF,5.0,2016-09-14 05:18:06.000,13900,3988,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 820..."
...,...,...,...,...,...,...,...
250960,AFV5AGWZ6DBOVMW46FMMI2NSQCVA,B00OUUXOUY,0.0,2016-06-23 01:00:28.000,7553,3585,"[843.0, 1601.0, 4169.0, 1691.0, 4546.0, 1814.0..."
217058,AFSFLSMTEC4M3I6EBPF72B4PLEPQ,B0047E8DN6,0.0,2015-09-05 23:31:51.000,7208,2500,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
323468,AEKDV6VVNKWG2WIXFOVCJBRORELQ,B0BZ5KPQZK,0.0,2018-05-27 21:31:44.724,2106,2818,"[667.0, 856.0, 984.0, 3151.0, 1499.0, 3774.0, ..."
132003,AEXSP5TFL7PDBHBCWOSLKXXMCMNA,B004VFH76A,0.0,2009-12-29 14:04:58.000,3839,1935,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."


In [80]:
with pd.option_context("display.max_colwidth", None):
    display(train_df.loc[train_df["user_id"] == "AEEV5YWQKPBTLFWHKOBBULYA2RDQ"].sort_values(by=args.timestamp_col, ascending=False))

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
213464,AEEV5YWQKPBTLFWHKOBBULYA2RDQ,B07C1RSV9C,0.0,2015-08-05 16:31:49,1412,4273,"[-1.0, -1.0, -1.0, 4559.0, 4443.0, 3164.0, 1047.0, 4685.0, 107.0, 3295.0]"
41296,AEEV5YWQKPBTLFWHKOBBULYA2RDQ,B07C1RSV9C,5.0,2015-08-05 16:31:49,1412,3276,"[-1.0, -1.0, -1.0, 4559.0, 4443.0, 3164.0, 1047.0, 4685.0, 107.0, 3295.0]"
213463,AEEV5YWQKPBTLFWHKOBBULYA2RDQ,B07C1RSV9C,0.0,2015-08-05 16:31:49,1412,2770,"[-1.0, -1.0, -1.0, 4559.0, 4443.0, 3164.0, 1047.0, 4685.0, 107.0, 3295.0]"
171822,AEEV5YWQKPBTLFWHKOBBULYA2RDQ,B07CB22VVJ,0.0,2014-07-17 19:19:28,1412,3078,"[-1.0, -1.0, -1.0, -1.0, 4559.0, 4443.0, 3164.0, 1047.0, 4685.0, 107.0]"
171821,AEEV5YWQKPBTLFWHKOBBULYA2RDQ,B07CB22VVJ,0.0,2014-07-17 19:19:28,1412,1496,"[-1.0, -1.0, -1.0, -1.0, 4559.0, 4443.0, 3164.0, 1047.0, 4685.0, 107.0]"
20475,AEEV5YWQKPBTLFWHKOBBULYA2RDQ,B07CB22VVJ,5.0,2014-07-17 19:19:28,1412,3295,"[-1.0, -1.0, -1.0, -1.0, 4559.0, 4443.0, 3164.0, 1047.0, 4685.0, 107.0]"
171820,AEEV5YWQKPBTLFWHKOBBULYA2RDQ,B000I23TTE,0.0,2014-07-17 19:16:43,1412,6,"[-1.0, -1.0, -1.0, -1.0, -1.0, 4559.0, 4443.0, 3164.0, 1047.0, 4685.0]"
171819,AEEV5YWQKPBTLFWHKOBBULYA2RDQ,B000I23TTE,0.0,2014-07-17 19:16:43,1412,4771,"[-1.0, -1.0, -1.0, -1.0, -1.0, 4559.0, 4443.0, 3164.0, 1047.0, 4685.0]"
20474,AEEV5YWQKPBTLFWHKOBBULYA2RDQ,B000I23TTE,5.0,2014-07-17 19:16:43,1412,107,"[-1.0, -1.0, -1.0, -1.0, -1.0, 4559.0, 4443.0, 3164.0, 1047.0, 4685.0]"
171818,AEEV5YWQKPBTLFWHKOBBULYA2RDQ,B0BYSP9676,0.0,2014-07-17 19:16:20,1412,1903,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 4559.0, 4443.0, 3164.0, 1047.0]"


## Convert user_id and item_id into indices

In [81]:
# idm_path = os.path.abspath("../data_for_ai/interim/idm_16407u.json")
# idm = IDMapper().load(idm_path)
# idm.get_user_id(1)

In [82]:
# train_df = train_df.pipe(idm.map_indices)
# val_df = val_df.pipe(idm.map_indices)

# assert idm.unknown_item_index not in train_df["item_indice"].values, "Unknown item index must be present in training data."
# assert idm.unknown_user_index not in train_df["user_indice"].values, "Unknown user index must be present in training data."
# assert idm.unknown_item_index not in val_df["item_indice"].values, "Unknown item index must be present in validation data."
# assert idm.unknown_user_index not in val_df["user_indice"].values, "Unknown user index must be present in validation data."

In [83]:
# train_df.head(3)

In [84]:
# assert train_df.groupby(args.user_col)[args.item_col].nunique().min() >= 5, "Each user must have at least five items."
# assert train_df.groupby(args.item_col)[args.user_col].nunique().min() >= 10, "Each item must have at least ten users."

## Training loop

In [85]:
rating_dataset = UserItemBinaryRatingDFDataset(
    train_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col, "item_sequence"
)
val_rating_dataset = UserItemBinaryRatingDFDataset(
    val_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col, "item_sequence"
)

train_loader = DataLoader(
    rating_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True
)
val_loader = DataLoader(
    val_rating_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False
)

In [86]:
item_indices = train_df[args.item_col].unique()
user_indices = train_df[args.user_col].unique()
n_items = len(item_indices)
n_users = len(user_indices)

logger.info(f"Number of users: {n_users}, Number of items: {n_items}")
model = init_model(n_users, n_items, args.embedding_dim, args.dropout)

2025-06-21 03:02:45.752 | INFO     | __main__:<module>:6 - Number of users: 16407, Number of items: 4817
/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:385: UserWarning:

enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.activation_relu_or_gelu was not True

2025-06-21 03:02:45.797 | INFO     | src.algo.sequence.model:__init__:118 - Start token used: 4816, Padding token used: 4817


In [87]:
idm_path = os.path.abspath("../data_for_ai/interim/idm_16407u.json")
idm = IDMapper().load(idm_path)
idm.get_user_id(1)

'AE227WAM4NWQPJI33OPN7ZARNNZQ'

## Overfit 1 batch

In [88]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, mode="min", verbose=False
)

model = init_model(n_users, n_items, args.embedding_dim, args.dropout)
lit_model = SeqModellingLitModule(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persit_dp,
    accelerator=args.device,
    idm= idm
)

log_dir = f"{args.notebook_persit_dp}/logs/overfit"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=300,
    overfit_batches=1,
    callbacks=[early_stopping],
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=train_loader,
)
logger.info(f"Logs available at {trainer.log_dir}")

2025-06-21 03:02:45.972 | INFO     | src.algo.sequence.model:__init__:118 - Start token used: 4816, Padding token used: 4817
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type                     | Params | Mode 
------------------------------------------------------------------------
0 | model              | SequenceRatingPrediction | 6.4 M  | train
1 | val_roc_auc_metric | BinaryAUROC              | 0      | train
2 | val_pr_auc_metric  | BinaryAveragePrecision   | 0      | train
------------------------------------------------------------------------
6.4 M     Trainable params
0         Non-trainable params
6.4 M     Total params
25.689    Total est

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:252: UserWarning:

You requested to overfit but enabled val dataloader shuffling. We are turning off the val dataloader shuffling for you.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:252: UserWarning:

You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/conne

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

2025-06-21 03:02:54.716 | INFO     | __main__:<module>:30 - Logs available at /home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/006-sequence-modelling-attn-256-dim-bce-prelu/logs/overfit/lightning_logs/version_14


In [89]:
for i in train_loader:
    print(i)
    break

{'user': tensor([ 8512, 12615,  2264, 11242,   914,  6131, 12624,  1059,  3250,  3115,
         6735,  1133, 15637, 14998, 14034,  7308,  8181,  9651, 16196,  3756,
        13708,  2419, 15171, 14030,  6298, 12269,  3622,  4953,   258,  8216,
         7581,  6873,  5484, 12819, 11752,  1673, 13128,  5780,  1204,  8917,
        12827, 15350,   505,  3314, 11497,  5003, 15973, 12756,  7398, 13538,
        10852,  1876, 14061,  7144, 14264,  3873,  2530, 10412, 13512,   604,
         5346,  7233,  5393, 10919,  8295,  5293,  7180, 13101, 13708,  8291,
         7958,   114,  6590,  8334,  8212, 15497,  3294,   894,  7286,  8032,
         2923, 13742, 14604,  7237,  4836,   475,   806, 11032,  9433, 15604,
         7270,  3781,  6984, 10155, 14383,  1938, 12778, 16228,  2462, 10694,
         7489,  1077,  3180,  4217,  1660,  3756,   268, 15712,  9974,  7635,
          455, 11137,  8696,  7397,  2459, 12173,  5704, 14470,  1892,  3306,
         5198,  8396,   760,  8681, 15618, 16356, 15029

## Run on all data

In [90]:
early_stopping = EarlyStopping(
    monitor="val_roc_auc", patience=args.early_stopping_patience, mode="max", verbose=False, min_delta=0.001
)

checkpoint_callback = ModelCheckpoint(
    dirpath=f"{args.notebook_persit_dp}/checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    monitor="val_roc_auc",
    mode="max",
)

model = init_model(n_users, n_items, args.embedding_dim, args.dropout)

print(f"Model: {model}")
lit_model = SeqModellingLitModule(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persit_dp,
    accelerator=args.device,
    idm= idm
)

log_dir = f"{args.notebook_persit_dp}/logs/run"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=args.max_epochs,
    callbacks=[early_stopping, checkpoint_callback],
    logger=args._mlf_logger if args.log_to_mlflow else None,
    
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    
)

# Change the library as a workaround for the issue in the latest Lightning release
#https://github.com/Lightning-AI/pytorch-lightning/pull/20669/commits/429f732a0528c558e701da7ec01e51c1e2e4f32e

2025-06-21 03:02:54.923 | INFO     | src.algo.sequence.model:__init__:118 - Start token used: 4816, Padding token used: 4817
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Model: SequenceRatingPrediction(
  (item_embedding): Embedding(4818, 256, padding_idx=4817)
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
    )
    (linear1): Linear(in_features=256, out_features=256, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (linear2): Linear(in_features=256, out_features=256, bias=True)
    (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.3, inplace=False)
    (dropout2): Dropout(p=0.3, inplace=False)
    (activation): PReLU(num_parameters=1)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: UserWarning:

Checkpoint directory /home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/006-sequence-modelling-attn-256-dim-bce-prelu/checkpoints exists and is not empty.

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type                     | Params | Mode 
------------------------------------------------------------------------
0 | model              | SequenceRatingPrediction | 6.4 M  | train
1 | val_roc_auc_metric | BinaryAUROC              | 0      | train
2 | val_pr_auc_metric  | BinaryAveragePrecision   | 0      | train
------------------------------------------------------------------------
6.4 M     Trainable params
0         Non-trainable params
6.4 M     Total params
25.689    Total estimated model params size (MB)
33        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

🏃 View run 006-sequence-modelling-attn-256-dim-bce-prelu at: http://138.2.61.6:5002/#/experiments/2/runs/579c4df80338486badd9fe6174ab1b88
🧪 View experiment at: http://138.2.61.6:5002/#/experiments/2


## Log metrics

In [91]:
logger.info(f"Loading best checkpoint from {checkpoint_callback.best_model_path}...")
args.best_checkpoint_path = checkpoint_callback.best_model_path

best_trainer = SeqModellingLitModule.load_from_checkpoint(
    checkpoint_path=checkpoint_callback.best_model_path,
    model=init_model(n_users, n_items, args.embedding_dim, args.dropout),
)

2025-06-21 03:24:36.407 | INFO     | __main__:<module>:1 - Loading best checkpoint from /home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/006-sequence-modelling-attn-256-dim-bce-prelu/checkpoints/best-checkpoint-v13.ckpt...
2025-06-21 03:24:36.447 | INFO     | src.algo.sequence.model:__init__:118 - Start token used: 4816, Padding token used: 4817


In [92]:
best_model = best_trainer.model.to(args.device)
best_model.eval()

SequenceRatingPrediction(
  (item_embedding): Embedding(4818, 256, padding_idx=4817)
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
    )
    (linear1): Linear(in_features=256, out_features=256, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (linear2): Linear(in_features=256, out_features=256, bias=True)
    (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.3, inplace=False)
    (dropout2): Dropout(p=0.3, inplace=False)
    (activation): PReLU(num_parameters=1)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_feat

In [93]:
val_recs_df = val_df.sort_values(by=args.timestamp_col).drop_duplicates(subset=[args.user_col], keep="first")

In [94]:
mlflow.start_run(run_id = trainer.logger.run_id)

<ActiveRun: >

### Classification metrics

In [95]:
val_user_indices = val_df["user_indice"].values
val_item_indices = val_df["item_indice"].values
val_item_sequences = val_df["item_sequence"].values.tolist()

In [96]:
users = torch.tensor(val_user_indices, device=args.device)
item_sequences = torch.tensor(val_item_sequences, device=args.device)
items = torch.tensor(val_item_indices, device=args.device)
classifications = best_model.predict(users, item_sequences, items)

OutOfMemoryError: CUDA out of memory. Tried to allocate 306.00 MiB. GPU 0 has a total capacity of 3.69 GiB of which 274.94 MiB is free. Process 101552 has 100.00 MiB memory in use. Process 105495 has 100.00 MiB memory in use. Process 107065 has 100.00 MiB memory in use. Process 108859 has 204.00 MiB memory in use. Process 412817 has 80.00 MiB memory in use. Process 412227 has 204.00 MiB memory in use. Process 452652 has 1.06 GiB memory in use. Including non-PyTorch memory, this process has 1.54 GiB memory in use. Of the allocated memory 1.40 GiB is allocated by PyTorch, and 41.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
classifications.shape

torch.Size([10437, 1])

In [ ]:
eval_classification_df = val_df.assign(
    classification_proba=classifications.cpu().detach().numpy(),
    label=lambda df: df[args.rating_col].gt(0).astype(int),
)

In [ ]:
eval_classification_df.head(3)

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence,classification_proba,label
129609,AGYYEQ2BPGILFTNVVZGEEK4AP5QA,B0854CTL5N,5.0,2021-08-10 01:19:51.147,12109,3857,"[-1, -1, 2822, 3056, 4462, 2519, 1791, 1396, 2...",0.834647,1
391795,AFSU64CATRPYNY6C6PCI6FGKCJ4Q,B0C1Z2FM57,0.0,2021-12-24 19:34:06.504,7275,3944,"[-1, 394, 3954, 1498, 2648, 3731, 4611, 1682, ...",0.301493,0
128128,AHP6PY4J6DB72JLOALVFWIOZ6EHA,B0B4SWZTZ1,1.0,2021-02-23 00:36:06.416,15029,4413,"[-1, -1, -1, 1280, 2811, 3304, 3153, 3188, 478...",0.823260,1


In [ ]:
classification_report = log_classification_metrics(
    args,
    eval_classification_df,
    target_col="label",
    prediction_col="classification_proba",
)

### Ranking metrics

In [ ]:
val_recs_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
127392,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B004FV4ROA,1.0,2020-12-27 00:30:31.146,11295,528,"[1715, 2537, 3743, 506, 4490, 3479, 3908, 2723..."
385658,AEHS7YR7BGGWMZS24H5UR5IP46HQ,B08F1P3BCC,0.0,2020-12-27 01:44:52.242,1784,4451,"[-1, -1, -1, -1, -1, 3382, 4330, 423, 3167, 2677]"
385660,AGAVHCK42EGMVS7DGPRX6HBCUCNQ,B09Q3NR84W,0.0,2020-12-27 02:25:48.357,9042,4650,"[-1, -1, -1, -1, 3104, 1416, 3743, 2694, 3612,..."
385662,AEFVBMCJAFNULDI5V2CKKTBCPURA,B07N1L5HX1,0.0,2020-12-27 02:32:15.171,1542,2677,"[-1, -1, -1, -1, -1, 1320, 2162, 2472, 2694, 3..."
385664,AGLXMKHBLTBNT3X2CLBAPW6QUTQA,B0BB6Y5N3M,0.0,2020-12-27 03:37:22.772,10418,1230,"[341, 3803, 4431, 1067, 4530, 4018, 2688, 4365..."
...,...,...,...,...,...,...,...
130864,AGGDNWGN3NDJ2DI5CBSFOMUAM6XA,B083YHS7SV,4.0,2022-02-18 19:43:25.492,9711,3844,"[-1, -1, -1, -1, 1019, 754, 2059, 413, 4262, 3..."
392604,AEKUF6AOVWDWFYOKPWO2CV72PEDQ,B07QN33986,0.0,2022-02-19 01:32:51.519,2171,1860,"[-1, -1, 2627, 4216, 4743, 1945, 2355, 1831, 9..."
392606,AFBTD25HPE4BE4LUFV3DTI2E2N2A,B07TMJ8S5Z,0.0,2022-02-19 16:49:57.966,5159,3411,"[-1, -1, -1, -1, 2260, 3517, 3609, 3495, 3625,..."
392609,AHLN6GKTKZE22AON34YAQXTGK63A,B0C682GZ5X,0.0,2022-02-19 17:28:55.519,14550,4201,"[-1, -1, -1, -1, -1, 1812, 4165, 4575, 4807, 374]"


In [ ]:
recommendations = best_model.recommend(
    torch.tensor(val_recs_df["user_indice"].values, device=args.device),
    torch.tensor(val_recs_df["item_sequence"].values.tolist(), device=args.device),
    k=args.top_K,
    batch_size=1)

Generating recommendations:   0%|          | 0/2424 [00:00<?, ?it/s]

In [ ]:
recommendations_df = pd.DataFrame(recommendations).pipe(
    create_rec_df, idm, args.user_col, args.item_col
)
recommendations_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,11295,3381,0.887306,1.0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B07F4P3JH7
1,11295,3188,0.873093,2.0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B0791TX5P5
2,11295,3557,0.869680,3.0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B07N86MCD2
3,11295,3558,0.868753,4.0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B07N8VFFNS
4,11295,3195,0.867032,5.0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B0792K2BK6
...,...,...,...,...,...,...
242395,2446,2811,0.775475,96.0,AEMYBWDN67IB5IBTMHLHN76V4QHQ,B01MTF2Z37
242396,2446,3037,0.775374,97.0,AEMYBWDN67IB5IBTMHLHN76V4QHQ,B074HVY1GR
242397,2446,3590,0.775267,98.0,AEMYBWDN67IB5IBTMHLHN76V4QHQ,B07PHQ93TV
242398,2446,4565,0.775224,99.0,AEMYBWDN67IB5IBTMHLHN76V4QHQ,B0BMK6DC5W


In [ ]:
label_df = create_label_df(
    val_df,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
    timestamp_col=args.timestamp_col,
)
label_df

,user_id,parent_asin,rating,rating_rank
128912,AG2EMAD6UILFF4ITMMKH2NEFTYHA,B0BZJGGX2T,5.0,1.0
127943,AHPI7N36W4JJYOAA6MBAGWTDF3FA,B072MKFNV6,4.0,1.0
129406,AGOBLEZGF5OSPDVTIMA3DPWAENGA,B07H65KP63,5.0,1.0
130173,AFXRWTDGQJIDOTGMCIZKH5QPK5KA,B00PKTU83U,5.0,1.0
128534,AHWDVLU5PTXMLD6PSJ2Z5Q3JP4OA,B0719SNR5N,3.0,1.0
...,...,...,...,...
386737,AEN2KQVSR5TWRXNQS3OTFT4EZQCA,B0BRT7XFM5,0.0,27.0
388052,AFKERAMSXU4MWO3H53R7DEFOHUVQ,B07G5YXYFL,0.0,27.0
388051,AFKERAMSXU4MWO3H53R7DEFOHUVQ,B07G5YXYFL,0.0,28.0
387885,AFKERAMSXU4MWO3H53R7DEFOHUVQ,B07ZZVX1F2,0.0,29.0


In [ ]:
eval_df = merge_recs_with_target(
    recommendations_df,
    label_df,
    k=args.top_K,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
)
eval_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
42,8.0,3381.0,0.838484,1,AE24AB4DW5KYK3F5DYOT5VPW2VLA,B07F4P3JH7,0,NaN
51,8.0,3476.0,0.829306,2,AE24AB4DW5KYK3F5DYOT5VPW2VLA,B07J2FGZSM,0,NaN
76,8.0,3908.0,0.826412,3,AE24AB4DW5KYK3F5DYOT5VPW2VLA,B08CLNX58K,0,NaN
11,8.0,2874.0,0.822509,4,AE24AB4DW5KYK3F5DYOT5VPW2VLA,B01NBTFNVA,0,NaN
33,8.0,3188.0,0.820618,5,AE24AB4DW5KYK3F5DYOT5VPW2VLA,B0791TX5P5,0,NaN
...,...,...,...,...,...,...,...,...
252450,16403.0,4415.0,0.782819,99,AHZZM7BCJAF2UEMMBHZCLXBB2SVA,B0B5153M52,0,NaN
252426,16403.0,3589.0,0.782705,100,AHZZM7BCJAF2UEMMBHZCLXBB2SVA,B07PHB491R,0,NaN
252388,NaN,NaN,NaN,101,AHZZM7BCJAF2UEMMBHZCLXBB2SVA,B075QC3TZY,1,1.0
252389,NaN,NaN,NaN,101,AHZZM7BCJAF2UEMMBHZCLXBB2SVA,B075QC3TZY,0,2.0


In [ ]:
ranking_report = log_ranking_metrics(args, eval_df)

In [ ]:
mlflow.end_run()

🏃 View run 006-sequence-modelling-attn-256-dim-bce-prelu at: http://138.2.61.6:5002/#/experiments/2/runs/babe0da54ed64ca19db3a2cc29c87e74
🧪 View experiment at: http://138.2.61.6:5002/#/experiments/2


## Clean up

In [ ]:
all_params = [args]

if args.log_to_mlflow:
    run_id = trainer.logger.run_id

    with mlflow.start_run(run_id=run_id):
        for params in all_params:
            params_dict = params.model_dump()
            params_ = dict()
            for k, v in params_dict.items():
                if k == "top_K":
                    k = "top_big_K"
                if k == "top_k":
                    k = "top_small_k"
                params_[f"{params.__repr_name__()}.{k}"] = v
            mlflow.log_params(params_)

🏃 View run 006-sequence-modelling-attn-256-dim-bce-prelu at: http://138.2.61.6:5002/#/experiments/2/runs/babe0da54ed64ca19db3a2cc29c87e74
🧪 View experiment at: http://138.2.61.6:5002/#/experiments/2


In [ ]:
val_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
129609,AGYYEQ2BPGILFTNVVZGEEK4AP5QA,B0854CTL5N,5.0,2021-08-10 01:19:51.147,12109,3857,"[-1, -1, 2822, 3056, 4462, 2519, 1791, 1396, 2..."
391795,AFSU64CATRPYNY6C6PCI6FGKCJ4Q,B0C1Z2FM57,0.0,2021-12-24 19:34:06.504,7275,3944,"[-1, 394, 3954, 1498, 2648, 3731, 4611, 1682, ..."
128128,AHP6PY4J6DB72JLOALVFWIOZ6EHA,B0B4SWZTZ1,1.0,2021-02-23 00:36:06.416,15029,4413,"[-1, -1, -1, 1280, 2811, 3304, 3153, 3188, 478..."
390725,AEWIG2ZQRJD6GWLETDOPW4SROWDQ,B008ALAAV0,0.0,2021-09-28 14:36:12.232,3651,1434,"[-1, -1, -1, -1, 57, 4097, 2367, 3578, 3202, 3..."
390937,AH7JQQJXJT7ZTDVHTBEVY4Z6KWXQ,B074P4MG1Q,0.0,2021-10-13 19:02:20.376,12922,4244,"[-1, 631, 172, 751, 2285, 4748, 3372, 3261, 27..."
...,...,...,...,...,...,...,...
385719,AG3SRHRTZRVUR5FDMYIH5KZERM2A,B08RLW7918,0.0,2020-12-30 01:31:04.861,8429,3096,"[-1, -1, 3744, 1451, 3576, 4608, 4219, 4621, 3..."
130708,AFAFYOKLVZYF2VM2VZ6H37ATHOOA,B0BPZFW1JH,5.0,2022-01-26 18:05:13.266,4968,4595,"[-1, 3326, 2830, 1420, 4007, 426, 1375, 1678, ..."
385697,AFTVQN7ZMD2BL2MXQAAUVFB5OLFA,B07GTGHQHB,0.0,2020-12-29 04:55:44.239,7417,1423,"[-1, -1, -1, -1, 1332, 3839, 3530, 2874, 3203,..."
386113,AHR2PRQPTUBBLQF6LY3CSFAU3TIA,B00KFD0SYK,0.0,2021-01-13 15:07:02.170,15249,3033,"[-1, -1, 3886, 2954, 2353, 1806, 3419, 4415, 1..."


In [ ]:
with pd.option_context("display.max_colwidth", None):
    display(val_df.loc[val_df[args.user_col] == "AFDFMCKM4YEZG7MSZQOIFQZBORHQ"].sort_values(by=args.timestamp_col, ascending=False))

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
386494,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B08D7638C8,0.0,2021-01-27 01:03:20.667,5348,2965,"[2123, 422, 1137, 4016, 3503, 1571, 158, 2110, 4089, 487]"
127811,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B08D7638C8,5.0,2021-01-27 01:03:20.667,5348,3914,"[2123, 422, 1137, 4016, 3503, 1571, 158, 2110, 4089, 487]"
386493,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B08D7638C8,0.0,2021-01-27 01:03:20.667,5348,4520,"[2123, 422, 1137, 4016, 3503, 1571, 158, 2110, 4089, 487]"
127486,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B0043T7FXE,5.0,2021-01-02 20:56:05.330,5348,487,"[115, 2123, 422, 1137, 4016, 3503, 1571, 158, 2110, 4089]"
385843,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B0043T7FXE,0.0,2021-01-02 20:56:05.330,5348,3982,"[115, 2123, 422, 1137, 4016, 3503, 1571, 158, 2110, 4089]"
385844,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B0043T7FXE,0.0,2021-01-02 20:56:05.330,5348,1973,"[115, 2123, 422, 1137, 4016, 3503, 1571, 158, 2110, 4089]"
385842,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B091ZHX225,0.0,2021-01-02 20:37:37.904,5348,4139,"[935, 115, 2123, 422, 1137, 4016, 3503, 1571, 158, 2110]"
385841,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B091ZHX225,0.0,2021-01-02 20:37:37.904,5348,2677,"[935, 115, 2123, 422, 1137, 4016, 3503, 1571, 158, 2110]"
127485,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B091ZHX225,5.0,2021-01-02 20:37:37.904,5348,4089,"[935, 115, 2123, 422, 1137, 4016, 3503, 1571, 158, 2110]"


In [ ]:
with pd.option_context("display.max_colwidth", None):
    display(train_df.loc[train_df[args.user_col] == "AFDFMCKM4YEZG7MSZQOIFQZBORHQ"].sort_values(by=args.timestamp_col, ascending=False))

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
286258,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B00WUIC2KG,0.0,2017-04-26 01:15:29,5348,1432,"[320.0, 935.0, 115.0, 2123.0, 422.0, 1137.0, 4016.0, 3503.0, 1571.0, 158.0]"
286257,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B00WUIC2KG,0.0,2017-04-26 01:15:29,5348,2437,"[320.0, 935.0, 115.0, 2123.0, 422.0, 1137.0, 4016.0, 3503.0, 1571.0, 158.0]"
77693,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B00WUIC2KG,5.0,2017-04-26 01:15:29,5348,2110,"[320.0, 935.0, 115.0, 2123.0, 422.0, 1137.0, 4016.0, 3503.0, 1571.0, 158.0]"
59436,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B000YB862E,5.0,2016-06-10 01:18:59,5348,158,"[550.0, 320.0, 935.0, 115.0, 2123.0, 422.0, 1137.0, 4016.0, 3503.0, 1571.0]"
249744,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B000YB862E,0.0,2016-06-10 01:18:59,5348,201,"[550.0, 320.0, 935.0, 115.0, 2123.0, 422.0, 1137.0, 4016.0, 3503.0, 1571.0]"
249743,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B000YB862E,0.0,2016-06-10 01:18:59,5348,163,"[550.0, 320.0, 935.0, 115.0, 2123.0, 422.0, 1137.0, 4016.0, 3503.0, 1571.0]"
225291,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B00IVPU7AO,0.0,2015-11-20 05:16:02,5348,2189,"[1730.0, 550.0, 320.0, 935.0, 115.0, 2123.0, 422.0, 1137.0, 4016.0, 3503.0]"
225292,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B00IVPU7AO,0.0,2015-11-20 05:16:02,5348,4335,"[1730.0, 550.0, 320.0, 935.0, 115.0, 2123.0, 422.0, 1137.0, 4016.0, 3503.0]"
47210,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B00IVPU7AO,5.0,2015-11-20 05:16:02,5348,1571,"[1730.0, 550.0, 320.0, 935.0, 115.0, 2123.0, 422.0, 1137.0, 4016.0, 3503.0]"
225290,AFDFMCKM4YEZG7MSZQOIFQZBORHQ,B07KFQFDNB,0.0,2015-11-20 05:08:14,5348,4692,"[455.0, 1730.0, 550.0, 320.0, 935.0, 115.0, 2123.0, 422.0, 1137.0, 4016.0]"
